In [ ]:
from functools import partial

import numpy as np

from src.qm_utils.lattice.lattice import Lattice2D
from src.acband import acband_form_factors, K_func1

from brillouin_zones import construct_brillouin_zones

/home/cwpark/acband-laughlin-instability/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


∣NK⟩ Tip: You can load logged data with nk.utils.history.HistoryDict.from_file(data.log).

In [2]:
N_s = 27
K = 0.4

sqrt3 = np.sqrt(3)

lB = 1.0
a_M = (((4 * np.pi) / sqrt3) ** 0.5) * lB

# fourier_resolution = 128
fourier_resolution = 256
G_radius = 64
V1 = 1.0
v1 = 3 * V1 * (a_M ** 4) / (4 * np.pi)

e1 = np.array([1, 0])
e2 = np.array([0, 1])
a1 = a_M * e2
a2 = a_M * ((-sqrt3 / 2) * e1 + (1 / 2) * e2)
lattice = Lattice2D(np.stack([a1, a2]))
recip_lattice = lattice.reciprocal()

bz = construct_brillouin_zones(lattice)

bz_N_s = bz[N_s]

A = N_s * lattice.unit_cell_area

def V(q):
    return -v1 * np.linalg.norm(q, axis=-1) ** 2
 
b1, b2 = lattice.reciprocal_lattice_vectors
b3 = -(b1 + b2)

K_func_args = (K, b1, b2, b3)
K_func = partial(K_func1, args=K_func_args)


start = time.time()
G_coords, ac_ff = acband_form_factors(
    bz[N_s],
    lB,
    K_func,
    fourier_resolution,
    G_radius=G_radius,
    pbar=True
)
end = time.time()

Computing AC band form factors: 100%|██████████| 16641/16641 [08:33<00:00, 32.42it/s]


In [3]:
%load_ext juliacall

Detected IPython. Loading juliacall extension. See https://juliapy.github.io/PythonCall.jl/stable/compat/#IPython


In [4]:
%%julia

include("julia_ref/code_00_Lattice.jl")
include("julia_ref/code_01_FiniteLattice.jl")
include("julia_ref/code_10_InteractionMatrix.jl")
include("julia_ref/code_20_ManyBodyHamiltonian.jl")

ManyBodyHamiltonian_MK_2V1B_2B_col (generic function with 1 method)

In [5]:
bz_N_s.n11, bz_N_s.n12, bz_N_s.n21, bz_N_s.n22

(np.int64(3), np.int64(3), np.int64(-6), np.int64(3))

In [10]:
%%julia

tl = TriangularLattice()
fl = FiniteLattice(tl, 3, 3, -6, 3)
Qs = FiniteReciprocalLattice(tl, 32)

**warning: FiniteLattice does not satisfy necessary arithmetic operations**


2977-element Vector{Vector{Float64}}:
 [0.0, 0.0]
 [7.255197456936871, 0.0]
 [3.6275987284684357, 6.283185307179586]
 [-3.6275987284684357, 6.283185307179586]
 [-7.255197456936871, -0.0]
 [-3.6275987284684357, -6.283185307179586]
 [3.6275987284684357, -6.283185307179586]
 [14.510394913873743, 0.0]
 [10.882796185405308, 6.283185307179586]
 [7.255197456936871, 12.566370614359172]
 ⋮
 [192.2627326088271, -56.548667764616276]
 [195.89033133729552, -50.26548245743669]
 [199.51793006576398, -43.982297150257104]
 [203.1455287942324, -37.69911184307752]
 [206.77312752270083, -31.41592653589793]
 [210.40072625116926, -25.132741228718345]
 [214.02832497963772, -18.84955592153876]
 [217.65592370810614, -12.566370614359172]
 [221.28352243657457, -6.283185307179586]

In [18]:
%%julia

ff_Lambda = FormFactors_IdealChernBand_1V(fl, Qs, 0.4, 0)

2977-element Vector{Matrix{ComplexF64}}:
 [0.9999999999999997 - 7.945636494545275e-34im 0.9088452378471862 - 7.006779047116583e-18im … 0.9999999999999997 - 7.945636494545275e-34im 0.9999999999999997 - 7.945636494545275e-34im; 0.908845237847186 + 5.265280031328313e-18im 1.0000000000000009 - 3.47622311212855e-18im … 0.908845237847186 + 5.265280031328313e-18im 0.908845237847186 + 5.265280031328313e-18im; … ; 0.9999999999999997 - 7.945636494545275e-34im 0.9088452378471862 - 7.006779047116583e-18im … 0.9999999999999997 - 7.945636494545275e-34im 0.9999999999999997 - 7.945636494545275e-34im; 0.9999999999999997 - 7.945636494545275e-34im 0.9088452378471862 - 7.006779047116583e-18im … 0.9999999999999997 - 7.945636494545275e-34im 0.9999999999999997 - 7.945636494545275e-34im]
 [-0.09965530882253845 - 3.588374316410151e-17im -0.22045077955836093 - 0.10503303264550606im … -0.09965530882253845 - 3.588374316410151e-17im -0.09965530882253845 - 3.588374316410151e-17im; 0.004722379907828721 - 0.040182658

In [19]:
Lambda_ref = %julia ff_Lambda

In [15]:
Lambda_ref = np.array(Lambda_ref)

/tmp/ipykernel_1640157/1984497794.py:1: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments. To learn more, see the migration guide https://numpy.org/devdocs/numpy_2_0_migration_guide.html#adapting-to-changes-in-the-copy-keyword
  Lambda_ref = np.array(Lambda_ref)


In [16]:
Lambda_ref.shape

(2977,)